In [2]:
#### 참고 링크
#### https://github.com/anishmadan23/semantic-segmentation-indian-driving-dataset/blob/master/Preprocessing.ipynb
####

import os 
import cv2
import sys
import json

import numpy as np
import skimage.io as io

from skimage.draw import polygon
from skimage import img_as_float

In [3]:
root = '../datasets/IDD_Segmentation'

In [4]:
def get_all_labels(root):
    labels = []
    final_labels = {}
    
    _, segmap_folder, img_folder,_  = os.listdir(root)
#     sub_path_train_img = os.path.join(root, img_folder, './train')
    sub_path_train_seg = os.path.join(root, segmap_folder, './train')

#     sub_path_val_img = os.path.join(root, img_folder, './val')
    sub_path_val_seg = os.path.join(root, segmap_folder, './val')
    
    # extract training labels
    for folder in os.listdir(sub_path_train_seg):
#         curr_img_folder = os.path.join(sub_path_train_img, folder)
        curr_seg_folder = os.path.join(sub_path_train_seg, folder)
        for file in os.listdir(curr_seg_folder):
            if 'json' not in file:
                continue
            f = open(os.path.join(curr_seg_folder, file), 'r')
            data = json.loads(f.read())
            for obj in data['objects']:
                if obj['label'] not in labels:
                    labels.append(obj['label'])

    # extract validation labels
    for folder in os.listdir(sub_path_val_seg):
#         curr_img_folder = os.path.join(sub_path_val_img, folder)
        curr_seg_folder = os.path.join(sub_path_val_seg, folder)
        for file in os.listdir(curr_seg_folder):
            if 'json' not in file:
                continue
            f = open(os.path.join(curr_seg_folder, file), 'r')
            data = json.loads(f.read())
            for obj in data['objects']:
                if obj['label'] not in labels:
                    labels.append(obj['label'])
                    
    for i in range(len(labels)):
        final_labels[labels[i]] = i
    return final_labels

In [6]:
labels = get_all_labels(root)
for k,v in labels.items():
    print(f'{k} : {v}')

sky : 0
road : 1
drivable fallback : 2
vegetation : 3
building : 4
billboard : 5
pole : 6
bus : 7
obs-str-bar-fallback : 8
wall : 9
person : 10
fence : 11
curb : 12
autorickshaw : 13
traffic sign : 14
bridge : 15
guard rail : 16
motorcycle : 17
truck : 18
vehicle fallback : 19
rider : 20
animal : 21
sidewalk : 22
car : 23
bicycle : 24
traffic light : 25
parking : 26
non-drivable fallback : 27
fallback background : 28
caravan : 29
polegroup : 30
out of roi : 31
tunnel : 32
rail track : 33
license plate : 34
trailer : 35
train : 36
rectification border : 37
ego vehicle : 38


In [7]:
# IDD 공식홈페이지 순서에 맞추기
labels['road']                  = 0
labels['parking']               = 1
labels['drivable fallback']     = 2
labels['sidewalk']              = 3
labels['rail track']            = 4
labels['non-drivable fallback'] = 5
labels['person']                = 6
labels['animal']                = 7
labels['rider']                 = 8
labels['motorcycle']            = 9
labels['bicycle']               = 10
labels['autorickshaw']          = 11
labels['car']                   = 12
labels['truck']                 = 13
labels['bus']                   = 14
labels['caravan']               = 15
labels['trailer']               = 16
labels['train']                 = 17
labels['vehicle fallback']      = 18
labels['curb']                  = 19
labels['wall']                  = 20
labels['fence']                 = 21
labels['guard rail']            = 22
labels['billboard']             = 23
labels['traffic sign']          = 24
labels['traffic light']         = 25
labels['pole']                  = 26
labels['pole group']            = 27
labels['obs-str-bar-fallback']  = 28
labels['building']              = 29
labels['bridge']                = 30
labels['tunnel']                = 31
labels['vegetation']            = 32
labels['sky']                   = 33
labels['fallback background']   = 34
labels['unlabeled']             = 35
labels['ego vehicle']           = 36
labels['rectification border']  = 37
labels['out of roi']            = 38
labels['license plate']         = 39
print(labels)

{'sky': 33, 'road': 0, 'drivable fallback': 2, 'vegetation': 32, 'building': 29, 'billboard': 23, 'pole': 26, 'bus': 14, 'obs-str-bar-fallback': 28, 'wall': 20, 'person': 6, 'fence': 21, 'curb': 19, 'autorickshaw': 11, 'traffic sign': 24, 'bridge': 30, 'guard rail': 22, 'motorcycle': 9, 'truck': 13, 'vehicle fallback': 18, 'rider': 8, 'animal': 7, 'sidewalk': 3, 'car': 12, 'bicycle': 10, 'traffic light': 25, 'parking': 1, 'non-drivable fallback': 5, 'fallback background': 34, 'caravan': 15, 'polegroup': 30, 'out of roi': 38, 'tunnel': 31, 'rail track': 4, 'license plate': 39, 'trailer': 16, 'train': 17, 'rectification border': 37, 'ego vehicle': 36, 'pole group': 27, 'unlabeled': 35}


In [8]:
def create_segmentation_maps(root, labels):
    _, segmap_folder, img_folder,_  = os.listdir(root)

    
    sub_path_train_img = os.path.join(root, img_folder, './train')
    sub_path_train_seg = os.path.join(root, segmap_folder, './train')

    sub_path_val_img = os.path.join(root, img_folder, './val')
    sub_path_val_seg = os.path.join(root, segmap_folder, './val')
    
    for folder in os.listdir(sub_path_train_seg):
        curr_img_folder = os.path.join(sub_path_train_img, folder)
        curr_seg_folder = os.path.join(sub_path_train_seg, folder)
        print(folder)
        for file in os.listdir(curr_img_folder):
            
            id = file.split('_')[0]
            #img = cv2.imread(os.path.join(curr_img_folder, file))
            
            f = open(os.path.join(curr_seg_folder, id+'_gtFine_polygons.json'), 'r')
            data = json.loads(f.read())
            seg_map = np.zeros((data['imgWidth'],data['imgHeight']))
            for obj in data['objects']:
                try : 
                    label = obj['label']
                    poly = np.array(obj['polygon'])
                    rr, cc = polygon(poly[:,0], poly[:,1], seg_map.shape)
                    seg_map[rr,cc] = labels[label]
                except : 
                    print(id)
                    print(label)
                    
                
            #cv2.imwrite(os.path.join(curr_img_folder, id+'.png'), img)
            cv2.imwrite(os.path.join(curr_seg_folder, id+'.png'), seg_map.T)

            
    for folder in os.listdir(sub_path_val_seg):
        curr_img_folder = os.path.join(sub_path_val_img, folder)
        curr_seg_folder = os.path.join(sub_path_val_seg, folder)
        print(folder)
        for file in os.listdir(curr_img_folder):
            id = file.split('_')[0]
            #img = cv2.imread(os.path.join(curr_img_folder, file))
            
            f = open(os.path.join(curr_seg_folder, id+'_gtFine_polygons.json'), 'r')
            data = json.loads(f.read())
            seg_map = np.zeros((data['imgWidth'],data['imgHeight']), np.uint8)
            for obj in data['objects']:
                try : 
                    label = obj['label']
                    poly = np.array(obj['polygon'])
                    rr, cc = polygon(poly[:,0], poly[:,1], seg_map.shape)
                    seg_map[rr,cc] = labels[label]
                except:
                    print(id)
                    print(label)
                    
            #cv2.imwrite(os.path.join(curr_img_folder, id+'.png'), img)
            cv2.imwrite(os.path.join(curr_seg_folder, id+'.png'), seg_map.T)

In [9]:
create_segmentation_maps(root, labels)

118
64
95
137
156
91
40
78
59
79
146
52
44
85
173
49
11
124
81
164
72
151
100
41
139
10
136
16
26
106
90
176
168
60
128
75
162
145
9
6
107
86
116
125
23
174
110
63
170
87
53
141
36
158
77
43
108
155
20
117
71
126
171
178
48
133
39
175
70
31
5
131
177
144
104
166
142
69
30
56
32
84
152
1
76
7
68
28
157
115
127
138
34
140
0
45
121
130
98
25
154
80
160
102
2
93
57
122
42
135
96
46
143
38
163
37
29
101
94
65
24
181
17
18
3
119
67
89
66
62
153
132
88
167
47
149
148
147
150
172
21
51
